In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#imports
import re

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import wordcloud
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
import string

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from collections import Counter
from collections import defaultdict
from plotly import tools
from plotly.subplots import make_subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [ ]:
#Reading Data
df_train = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
df_test = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')

### **Meta features Creation**
Based on this SRK's EDA [kernal](https://www.kaggle.com/sudalairajkumar/simple-feature-engg-notebook-spooky-author), creating following meta-features:-
1. Length of test
2. Number of words in the text
3. Number of unique words in the text
4. Number of characters in the text
5. Number of stopwords
6. Number of punctuations
7. Number of upper case words
8. Number of title case words
9. Number of numericals in the text
10. Average length of the words
11. Check for urls

The idea behind creating above features is to check if it helps in identifing distinction b/w negatives and positives

In [ ]:
class metaFeatures:
    
    def __init__(self,df):
        self.df = df
    
    #Number of words in the text
    @staticmethod
    def num_of_words(ech_row):
        rtn = len(ech_row.split())
        return rtn
    
    #Number of unique words in the text
    @staticmethod
    def num_of_unqwords(ech_row):
        rtn = len(set(ech_row.split()))
        return rtn
    
    #Number of characters in the text
    @staticmethod
    def num_of_chars(ech_row):
        rtn = len(set(ech_row))
        return rtn
    
    #Number of stopwords
    @staticmethod
    def num_of_stopwords(ech_row):
        stopwords_lst = list(stopwords.words('english'))
        rtn = len([s for s in str(ech_row).lower().split() if s in stopwords_lst])
        return rtn
    
    #Number of punctuations
    @staticmethod
    def num_of_punctuations(ech_row):
        punc_lst = list(string.punctuation)
        rtn = len([p for p in str(ech_row).lower().split() if p in punc_lst])
        return rtn
    
    #Number of upper case words
    @staticmethod
    def num_of_uppercase(ech_row):
        rtn = len([p for p in str(ech_row).split() if p.isupper()])
        return rtn
    
    #Number of title case words
    @staticmethod
    def num_of_titlecase(ech_row):
        rtn = len([p for p in str(ech_row).split() if p.istitle()])
        return rtn
    
    #Number of numericals in the text
    @staticmethod
    def num_of_numericals(ech_row):
        numer_lst = ['0','1','2','3','4','5','6','7','8','9']
        rtn = len([p for p in numer_lst if p in ech_row])
        return rtn
    
    #Average length of the words
    @staticmethod
    def words_avglen(ech_row):
        rtn = np.round( np.mean([len(p) for p in str(ech_row).split()]) ,2)
        return rtn
    
    # URLs Check
    @staticmethod
    def urls_count(ech_row):
        rtn = len([h for h in str(ech_row).lower().split() if 'http' in h or 'https' in h])
        return rtn
    
    #final calculations
    def calc(self):
        
        self.df['text_len'] = self.df['question_text'].apply(lambda x: len(x))
        self.df['num_of_words'] = self.df['question_text'].apply(lambda x: self.num_of_words(x))
        self.df['num_of_unqwords'] = self.df['question_text'].apply(lambda x: self.num_of_unqwords(x))
        self.df['num_of_chars'] = self.df['question_text'].apply(lambda x: self.num_of_chars(x))
        self.df['num_of_stopwords'] = self.df['question_text'].apply(lambda x: self.num_of_stopwords(x))
        self.df['num_of_punctuations'] = self.df['question_text'].apply(lambda x: self.num_of_punctuations(x))
        self.df['num_of_uppercase'] = self.df['question_text'].apply(lambda x: self.num_of_uppercase(x))
        self.df['num_of_titlecase'] = self.df['question_text'].apply(lambda x: self.num_of_titlecase(x))
        self.df['num_of_numericals'] = self.df['question_text'].apply(lambda x: self.num_of_numericals(x))
        self.df['words_avglen'] = self.df['question_text'].apply(lambda x: self.words_avglen(x))
        self.df['urls_count'] = self.df['question_text'].apply(lambda x: self.urls_count(x))
        
        return self.df

metafeatures = metaFeatures(df_train)
df_train_feat = metafeatures.calc()

#positive and negative reviews
df_train_pos = df_train_feat[df_train_feat.target==1]
df_train_neg = df_train_feat[df_train_feat.target==0]

df_train_feat.head()

### **Data Cleaning**

In [ ]:
nltk.download('wordnet')
class CleanIt:
    
    def __init__(self,df):
        self.df = df
    
    #Removes Punctuations
    @staticmethod
    def remove_punctuations(ech_row):
        punct_tag = re.compile(r'[^\w\s]')
        rtn = punct_tag.sub(r'',ech_row)
        return rtn
    
    #Removes HTML syntaxes
    @staticmethod
    def remove_html(ech_row):
        html_tag=re.compile(r'<.*?>')
        rtn=html_tag.sub(r'',ech_row)
        return rtn
    
    #Removes URL data
    @staticmethod
    def remove_url(ech_row):
        url_clean= re.compile(r"https://\S+|www\.\S+")
        rtn=url_clean.sub(r'',ech_row)
        return rtn
    
    #Removes Emojis
    @staticmethod
    def remove_emoji(ech_row):
        emoji_clean= re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
        rtn=emoji_clean.sub(r'',ech_row)
        url_clean= re.compile(r"https://\S+|www\.\S+")
        rtn=url_clean.sub(r'',rtn)
        return rtn
    
    #Lower Case
    @staticmethod
    def make_it_lower(ech_row):
        rtn=ech_row.lower()
        return rtn
    
    #Remove extra spaces
    @staticmethod
    def remove_extraSpace(ech_row):
        rtn=' '.join(ech_row.split())
        return rtn
    
    #Replace abbreviated pronouns with full forms
    @staticmethod
    def remove_abb(data):
        data = re.sub(r"he's", "he is", data)
        data = re.sub(r"there's", "there is", data)
        data = re.sub(r"We're", "We are", data)
        data = re.sub(r"That's", "That is", data)
        data = re.sub(r"won't", "will not", data)
        data = re.sub(r"they're", "they are", data)
        data = re.sub(r"Can't", "Cannot", data)
        data = re.sub(r"wasn't", "was not", data)
        data = re.sub(r"don\x89Ûªt", "do not", data)
        data= re.sub(r"aren't", "are not", data)
        data = re.sub(r"isn't", "is not", data)
        data = re.sub(r"What's", "What is", data)
        data = re.sub(r"haven't", "have not", data)
        data = re.sub(r"hasn't", "has not", data)
        data = re.sub(r"There's", "There is", data)
        data = re.sub(r"He's", "He is", data)
        data = re.sub(r"It's", "It is", data)
        data = re.sub(r"You're", "You are", data)
        data = re.sub(r"I'M", "I am", data)
        data = re.sub(r"shouldn't", "should not", data)
        data = re.sub(r"wouldn't", "would not", data)
        data = re.sub(r"i'm", "I am", data)
        data = re.sub(r"I\x89Ûªm", "I am", data)
        data = re.sub(r"I'm", "I am", data)
        data = re.sub(r"Isn't", "is not", data)
        data = re.sub(r"Here's", "Here is", data)
        data = re.sub(r"you've", "you have", data)
        data = re.sub(r"you\x89Ûªve", "you have", data)
        data = re.sub(r"we're", "we are", data)
        data = re.sub(r"what's", "what is", data)
        data = re.sub(r"couldn't", "could not", data)
        data = re.sub(r"we've", "we have", data)
        data = re.sub(r"it\x89Ûªs", "it is", data)
        data = re.sub(r"doesn\x89Ûªt", "does not", data)
        data = re.sub(r"It\x89Ûªs", "It is", data)
        data = re.sub(r"Here\x89Ûªs", "Here is", data)
        data = re.sub(r"who's", "who is", data)
        data = re.sub(r"I\x89Ûªve", "I have", data)
        data = re.sub(r"y'all", "you all", data)
        data = re.sub(r"can\x89Ûªt", "cannot", data)
        data = re.sub(r"would've", "would have", data)
        data = re.sub(r"it'll", "it will", data)
        data = re.sub(r"we'll", "we will", data)
        data = re.sub(r"wouldn\x89Ûªt", "would not", data)
        data = re.sub(r"We've", "We have", data)
        data = re.sub(r"he'll", "he will", data)
        data = re.sub(r"Y'all", "You all", data)
        data = re.sub(r"Weren't", "Were not", data)
        data = re.sub(r"Didn't", "Did not", data)
        data = re.sub(r"they'll", "they will", data)
        data = re.sub(r"they'd", "they would", data)
        data = re.sub(r"DON'T", "DO NOT", data)
        data = re.sub(r"That\x89Ûªs", "That is", data)
        data = re.sub(r"they've", "they have", data)
        data = re.sub(r"i'd", "I would", data)
        data = re.sub(r"should've", "should have", data)
        data = re.sub(r"You\x89Ûªre", "You are", data)
        data = re.sub(r"where's", "where is", data)
        data = re.sub(r"Don\x89Ûªt", "Do not", data)
        data = re.sub(r"we'd", "we would", data)
        data = re.sub(r"i'll", "I will", data)
        data = re.sub(r"weren't", "were not", data)
        data = re.sub(r"They're", "They are", data)
        data = re.sub(r"Can\x89Ûªt", "Cannot", data)
        data = re.sub(r"you\x89Ûªll", "you will", data)
        data = re.sub(r"I\x89Ûªd", "I would", data)
        data = re.sub(r"let's", "let us", data)
        data = re.sub(r"it's", "it is", data)
        data = re.sub(r"can't", "cannot", data)
        data = re.sub(r"dont", "do not", data)
        data = re.sub(r"don't", "do not", data)
        data = re.sub(r"you're", "you are", data)
        data = re.sub(r"i've", "I have", data)
        data = re.sub(r"that's", "that is", data)
        data = re.sub(r"i'll", "I will", data)
        data = re.sub(r"doesn't", "does not",data)
        data = re.sub(r"i'd", "I would", data)
        data = re.sub(r"didn't", "did not", data)
        data = re.sub(r"ain't", "am not", data)
        data = re.sub(r"you'll", "you will", data)
        data = re.sub(r"I've", "I have", data)
        data = re.sub(r"Don't", "do not", data)
        data = re.sub(r"I'll", "I will", data)
        data = re.sub(r"I'd", "I would", data)
        data = re.sub(r"Let's", "Let us", data)
        data = re.sub(r"you'd", "You would", data)
        data = re.sub(r"It's", "It is", data)
        data = re.sub(r"Ain't", "am not", data)
        data = re.sub(r"Haven't", "Have not", data)
        data = re.sub(r"Could've", "Could have", data)
        data = re.sub(r"youve", "you have", data)  
        data = re.sub(r"donå«t", "do not", data)
        return data
    
    # NLTK lemmatizer
    @staticmethod
    def lemma_traincorpus(ech_row):
        lemmatizer=WordNetLemmatizer()
        out_data=""
        for words in ech_row:
            out_data+= lemmatizer.lemmatize(words)
        return out_data

    #final calculations
    def calc(self):
        self.df['question_text'] = self.df['question_text'].apply(lambda x: self.remove_extraSpace(x))
        
        self.df['question_text'] = self.df['question_text'].apply(lambda x: self.remove_punctuations(x))
        self.df['question_text'] = self.df['question_text'].apply(lambda x: self.remove_html(x))
        self.df['question_text'] = self.df['question_text'].apply(lambda x: self.remove_url(x))
        self.df['question_text'] = self.df['question_text'].apply(lambda x: self.remove_emoji(x))
        self.df['question_text'] = self.df['question_text'].apply(lambda x: self.remove_abb(x))
        
        self.df['question_text'] = self.df['question_text'].apply(lambda x: self.make_it_lower(x))
        self.df['question_text'] = self.df['question_text'].apply(lambda x: self.lemma_traincorpus(x))
        return self.df

# import re
# data_clean = CleanIt(df_train)
# df_train_clean_feat = data_clean.calc()

# _lst = list(df_train_clean_feat['question_text'].sample(10))
# for x in _lst:
#     print(x)

### **TF-IDF**

Idea here is to represent text using TF-IDF tokens 

In [ ]:
%%time
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import ADASYN,SMOTE
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

## split to train and val
_train_df, val_df = train_test_split(df_train_clean_feat, test_size=0.2, random_state=202)

sample_train_df = _train_df

#tfidf
tfidf_vect=TfidfVectorizer(stop_words='english',ngram_range=(1, 2), lowercase=True, max_features=150000)
train_tfidf=tfidf_vect.fit_transform(sample_train_df['question_text'])
val_tfidf=tfidf_vect.transform(val_df['question_text'])


#Balancing The Sample for TFIDF Baseline
#SMOTE oversampling
val_x = val_tfidf
val_y = val_df['target']

smote=SMOTE(random_state=42)
smote_train_x,smote_train_y=smote.fit_sample(train_tfidf,sample_train_df['target'])
smote_train_x.shape,smote_train_y.shape
print(smote_train_x.shape,smote_train_y.shape,val_x.shape,val_y.shape)

### **ML Models**

For baseline score we are using logistic, xgboost and lightGBM

In [ ]:
%%time
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from collections import Counter
from sklearn.model_selection import StratifiedKFold,KFold,StratifiedShuffleSplit
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestRegressor,GradientBoostingClassifier,RandomForestClassifier

class Run_models:
    #imports

    def __init__(self,train_x,train_y,val_x,val_y):
        self.train_x = train_x
        self.train_y = train_y
        self.val_x = val_x
        self.val_y = val_y
    
    def kFold(self,mdl):

        kfold=StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
        rtn1=list(np.round(cross_val_score(mdl,self.train_x,self.train_y,cv=kfold,scoring='roc_auc'),3))
        pred_val_y=cross_val_predict(mdl,self.val_x,self.val_y)
        #pred_val_y = mdl.predict(val_x)
        rtn2= np.round(roc_auc_score(self.val_y, pred_val_y),3)
        rtn = {'on_train':rtn1,'on_val':rtn2}
        return rtn
        
    def logisticRegression(self):
        mdl = LogisticRegression(max_iter=1000,penalty='l2')
        rtn = self.kFold(mdl)
        return rtn
    
    
    def gradientBoost(self):
        mdl = GradientBoostingClassifier(learning_rate=0.02, loss='deviance',n_estimators=100)
        rtn = self.kFold(mdl)
        return rtn
    
    def xgBoost(self):
        mdl = XGBClassifier(n_estimators=50,random_state=42)
        rtn = self.kFold(mdl)
        return rtn
                            
    def lightGBM(self):
        mdl = LGBMClassifier(n_estimators=100,random_state=42)
        rtn = self.kFold(mdl)
        return rtn
    
    def apply_all_models(self):
        cv_rtn = {}
        #cv_rtn['LogisticRregression'] = self.logisticRegression()
        cv_rtn['gradientBoost'] = self.gradientBoost()
        #cv_rtn['xgBoost'] = self.xgBoost()
        cv_rtn['lightGBM'] = self.lightGBM()
        return cv_rtn

In [ ]:
# %%time

# models_all = Run_models(smote_train_x,smote_train_y,val_x,val_y)
# ml_tfidf_result = models_all.apply_all_models()
# print(ml_tfidf_result)

### **ML Model Results**

In [ ]:
%%time
models_all = Run_models(smote_train_x,smote_train_y,val_x,val_y)
ml_tfidf_result = models_all.apply_all_models()

for x in ml_tfidf_result.keys():
    print(x,ml_tfidf_result[x])

`Looks like model is over fitting train data as it performs badly on validation set`

**Using Embeddings**

In [ ]:
#imports
import re

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import wordcloud
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
import string

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from collections import Counter
from collections import defaultdict
from plotly import tools
from plotly.subplots import make_subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

#Reading Data
df_train = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
df_test = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')

Refered how to unzip embeddings file from [here](https://www.kaggle.com/c/quora-insincere-questions-classification/discussion/140385)

In [ ]:
import zipfile

Dataset = "embeddings"
with zipfile.ZipFile("../input/quora-insincere-questions-classification/"+Dataset+".zip","r") as z:
     z.extractall(".")

from subprocess import check_output
print(check_output(["ls", "."]).decode("utf8"))

In [ ]:
%%time
from gensim.models import Word2Vec,KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

google_loaded='./GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
google_model = KeyedVectors.load_word2vec_format(google_loaded, binary=True)
plt.plot(google_model['reviews'])
plt.plot(google_model['injustice'])
plt.show()

In [ ]:
import re
data_clean = CleanIt(df_train)
df_train_clean_feat = data_clean.calc()

In [ ]:
import sklearn
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import ADASYN,SMOTE
from tensorflow.keras.preprocessing.text import Tokenizer

## split to train and val
token_train_df, token_val_df = train_test_split(df_train_clean_feat, test_size=0.1, random_state=202)

num_words = None # how many unique words to use 
fltrs='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n' # characters that will be filtered from the texts

# Tokenize the sentences
tokenizer = Tokenizer(num_words = None,filters=fltrs,lower=True)
tokenizer.fit_on_texts(list(df_train.question_text) + list(df_test.question_text))


X = tokenizer.texts_to_sequences(token_train_df.question_text)
X_test = tokenizer.texts_to_sequences(token_val_df.question_text)

# Create vocabulary from all words
vocabulary = tokenizer.word_index

del tokenizer
import gc; gc.collect() #manual garbage collection


print(f"Tokenized sample text: {X[0]}\n")
print({word: vocabulary[word.lower()] for word in (token_train_df.iloc[0].question_text).split(' ')[:-1]})

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LENGTH = 50 # Max. number of words in sentence
pad = 'pre' # Where to add padding

X_embed = pad_sequences(X, maxlen=MAX_LENGTH, padding=pad)
X_test_embed = pad_sequences(X_test,maxlen=MAX_LENGTH, padding=pad)

print(f"Padded sample text: \n{X[0]}")

In [ ]:
google_in_vocab = {}
google_out_vocab = {}

for w in vocabulary.keys():
    if w in google_model:
        w = w.lower()
        google_in_vocab[w] = google_model[w]
    else:
        google_out_vocab[w]=0


coverage = np.round(len(google_in_vocab)*100/len(vocabulary),2)
print(f'% coverage og google embeddings is : {coverage} %')

In [ ]:
# Delete unused embeddings
del google_model
gc.collect()

### **ML. Models on Google News Embeddings**

From google news embeddings we are trying to aggrigate word embeddings of each sentence in `question_text`

In [ ]:
def convert_sentence_embeddings(data,_vocab):
    vocab=[w.lower() for w in data.split(' ') if w.lower() in _vocab.keys()]
    if len(vocab)>0:
        avg_pool=np.mean([_vocab[x] for x in vocab],axis=0)
    else:
        avg_pool = np.zeros(shape=(300,))
    return avg_pool

#_train_df, val_df, sampeling only 10000 due to memory issues
train_google_df = token_train_df
val_google_df = token_val_df
train_google_df['google_news_vec']=token_train_df['question_text'].apply(lambda z: convert_sentence_embeddings(z,google_in_vocab) )
val_google_df['google_news_vec']= token_val_df['question_text'].apply(lambda z: convert_sentence_embeddings(z,google_in_vocab) )

google_train_df = train_google_df #[train_google_df.need_col != np.nan]
google_val_df = val_google_df #[val_google_df.need_col != np.nan]




# #SMOTE oversampling
# val_google_x = google_val_df['google_news_vec']
# val_google_y = google_val_df['target']

# smote=SMOTE(random_state=42)
# smote_google_train_x,smote_google_train_y=smote.fit_sample(google_train_df['google_news_vec'],google_train_df['target'])
# print(smote_google_train_x.shape,smote_google_train_y.shape,val_google_x.shape,val_google_y.shape)

### **ML Model Results**

In [ ]:
train_google_df.head()

In [ ]:
list(train_google_df['google_news_vec'].head())

In [ ]:
%%time

#X
train_google_x = list(train_google_df['google_news_vec'])
val_google_x = list(val_google_df['google_news_vec'])
#Y
train_google_y = list(train_google_df['target'])
val_google_y = list(val_google_df['target'])

models_all = Run_models(train_google_x,train_google_y,val_google_x,val_google_y)
ml_tfidf_result = models_all.apply_all_models()

for x in ml_tfidf_result.keys():
    print(x,ml_tfidf_result[x])

In [ ]:
%%time

#X
train_google_x = list(train_google_df['google_news_vec'])
val_google_x = list(val_google_df['google_news_vec'])
#Y
train_google_y = list(train_google_df['target'])
val_google_y = list(val_google_df['target'])

models_all = Run_models(train_google_x,train_google_y,val_google_x,val_google_y)
ml_tfidf_result = models_all.apply_all_models()

for x in ml_tfidf_result.keys():
    print(x,ml_tfidf_result[x])